In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2, os
import threading
import numpy as np
import torch
from torchvision import models, transforms
from sklearn.cluster import DBSCAN
from ultralytics import YOLO

class VideoPlayer:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Player - mobilenet")

        # Create frames for layout
        self.left_frame = tk.Frame(root)
        self.left_frame.grid(row=0, column=0, padx=10, pady=10)

        self.right_frame = tk.Frame(root)
        self.right_frame.grid(row=0, column=1, padx=10, pady=10)

        # Video display canvas
        self.canvas = tk.Canvas(self.left_frame, width=800, height=480, bg="grey")
        self.canvas.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

        self.load_button = tk.Button(self.left_frame, text="Select a Video", command=self.load_video)
        self.load_button.grid(row=1, column=0, columnspan=2, pady=10)

        # Progress bar
        self.progress = tk.Scale(self.left_frame, from_=0, to=100, orient=tk.HORIZONTAL, length=800, label="Progress")
        self.progress.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

        # Controls for team visibility
        self.effects_label = tk.Label(self.right_frame, text="Effects on")
        self.effects_label.grid(row=0, column=0, pady=10)

        self.team_a_var = tk.BooleanVar(value=True)
        self.team_b_var = tk.BooleanVar(value=True)
        self.hoop_var = tk.BooleanVar(value=True)
        self.ball_var = tk.BooleanVar(value=True)

        self.team_a_check = tk.Checkbutton(self.right_frame, text="Team A", variable=self.team_a_var)
        self.team_a_check.grid(row=1, column=0, sticky="w", padx=5, pady=5)

        self.team_b_check = tk.Checkbutton(self.right_frame, text="Team B", variable=self.team_b_var)
        self.team_b_check.grid(row=2, column=0, sticky="w", padx=5, pady=5)

        self.hoop_check = tk.Checkbutton(self.right_frame, text="Hoop", variable=self.hoop_var)
        self.hoop_check.grid(row=3, column=0, sticky="w", padx=5, pady=5)

        self.ball_check = tk.Checkbutton(self.right_frame, text="Ball", variable=self.ball_var)
        self.ball_check.grid(row=4, column=0, sticky="w", padx=5, pady=5)

        self.video_path = None
        self.cap = None
        self.playing = False
        self.paused = False
        self.stop_flag = False

        # YOLOv8 model path
        current_dir = os.getcwd()
        model_path = os.path.join(current_dir, 'yolov8_trained2_0719.pt')
        self.model1 = YOLO(model_path)
        self.model1.model.names = {0: 'Ball', 1: 'Hoop', 2: 'Player'}
        
        # Load MobileNetV2 model
        self.mobilenet = models.mobilenet_v3_small(weights='MobileNet_V3_Small_Weights.IMAGENET1K_V1')
        self.mobilenet = torch.nn.Sequential(*list(self.mobilenet.children())[:-1])  # Remove last fully connected layer
        self.mobilenet.eval()

        # Define image preprocessing steps
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def toggle_all(self):
        state = self.all_var.get()
        self.team_a_var.set(state)
        self.team_b_var.set(state)

    def load_video(self):
        self.video_path = filedialog.askopenfilename()
        if self.video_path:
            self.cap = cv2.VideoCapture(self.video_path)
            self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total frame count
            self.progress.config(from_=0, to=100)  # Set progress bar range to 0-100%
            self.play_video()
        else:
            messagebox.showerror("Error", "Failed to Load Video")

    def play_video(self):
        if self.video_path and not self.playing:
            self.playing = True
            self.paused = False
            self.stop_flag = False
            threading.Thread(target=self._play).start()
        elif self.paused:
            self.paused = False

    def _play(self):
        while self.cap.isOpened() and self.playing and not self.stop_flag:
            if not self.paused:
                ret, frame = self.cap.read()
                if not ret:
                    break

                # Update progress bar as a percentage
                current_frame = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
                percentage = int((current_frame / self.total_frames) * 100)
                self.progress.set(percentage)

                # Process frame for displaying team bounding boxes
                frame = self.process_frame(frame)

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (800, 480))
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
                self.canvas.image = imgtk
                self.root.update()

                # Control playback speed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        self.playing = False
        self.cap.release()

    def extract_features(self, image):
        image_tensor = self.preprocess(image).unsqueeze(0)
        with torch.no_grad():
            features = self.mobilenet(image_tensor)
        return features.squeeze().numpy()

    def process_image(self, img):
        results = self.model1.predict(img)[0]
        
        features_list = []
        for result in results.boxes:
            class_id = int(result.cls[0])
            
            if class_id == 2:  # player
                x1, y1, x2, y2 = map(int, result.xyxy[0])
                cropped_img = img[y1:y2, x1:x2]
                cropped_pil_img = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
                features = self.extract_features(cropped_pil_img)
                features_list.append(features)
        
        n = len(features_list)
        if n < 2:
            print("特徵向量數量不足，無法進行聚類。")
            labels = []
        else:
            features_2d = np.array([f.flatten() for f in features_list])  # Ensure it's a 2D array
            
            # Perform DBSCAN clustering
            dbscan = DBSCAN(eps=0.9, min_samples=1, metric='euclidean').fit(features_2d)
            labels = dbscan.labels_
    
        colors = [(255, 0, 0), (0, 0, 255)]  # Red for B隊, Blue for A隊
        color_mapping = {}
        
        for label in set(labels):
            if label != -1 and label not in color_mapping:
                color_mapping[label] = colors[len(color_mapping) % len(colors)]
    
        j = 0
        for result in results.boxes:
            x1, y1, x2, y2 = map(int, result.xyxy[0])
            class_id = int(result.cls[0])
            score = float(result.conf[0])
            
            if (class_id == 1 and self.hoop_var.get()):  # Hoop
                color = (0, 255, 0)  # Green box
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f'{self.model1.model.names[class_id]} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            elif class_id == 2:  # Player
                if j < len(labels):
                    label = labels[j]
                    if label == -1:
                        color = (128, 128, 128)  # Gray for unclassified players
                    else:
                        color = color_mapping[label]
                    team_label = '(A)' if color == colors[1] else '(B)'
                    j += 1

                    # Add conditions to show/hide boxes based on team selection
                    if (team_label == '(A)' and self.team_a_var.get()) or (team_label == '(B)' and self.team_b_var.get()):
                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(img, f'{self.model1.model.names[class_id]} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                    elif not self.team_a_var.get() and not self.team_b_var.get():
                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(img, f'{self.model1.model.names[class_id]} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            elif (class_id == 0 and self.ball_var.get()):  # Ball
                color = (255, 255, 255)  # Green box
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f'{self.model1.model.names[class_id]} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
        return img

    def process_frame(self, frame):
        # Process current frame
        frame = self.process_image(frame)
        return frame

    def pause_video(self):
        if self.playing:
            self.paused = True

    def stop_video(self):
        if self.playing:
            self.stop_flag = True
            self.playing = False
            self.cap.release()
            self.canvas.delete("all")
            self.progress.set(0)  # Reset progress bar

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoPlayer(root)
    root.mainloop()
